In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# 1000 lines of labeled reviews to train
train_data = pd.read_csv('./data/train.tsv', delimiter='\t', nrows=1000)

train_data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
# 1000 lines of unlabeled reviews to test
test_data = pd.read_csv('./data/test.tsv', delimiter='\t', nrows=1000)

test_data.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...
